<b>Preprocessing</b>

In [ ]:
import pandas as pd
import glob

# Allows for printing all columns and increases width; debug utility
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Select only the data files in the data folder
#data_files = glob.glob('data/*data.csv')
data_files = sorted(glob.glob('data/*data.csv'))

# Create the dfs and merge them on the keys TIME and METROREG
merged_data = None
for file in data_files:
    current_df = pd.read_csv(file, encoding="ISO-8859-1")
    if merged_data is not None:
        merged_data = pd.merge(merged_data, current_df, on=["TIME", "METROREG"])
    else:
        merged_data = current_df

# Rename relevant columns and drop the useless ones
# Though, be aware that measurement information is also lost
renames = {'Value_x': 'employed_persons',
           'Value_y': 'gdp',
           'Value': 'population'}

droppes = ['UNIT_x', 'WSTATUS', 'NACE_R2', 'Flag and Footnotes_x',
           'UNIT_y', 'Flag and Footnotes_y', 'AGE', 'SEX', 'Flag and Footnotes']

filtered_data = merged_data.rename(columns=renames).drop(columns=droppes)

#print(filtered_data)

In [ ]:
#Rows we want to remove
non_metro=['Non-metropolitan regions in Austria',
 'Non-metropolitan regions in Belgium',
 'Non-metropolitan regions in Bulgaria',
 'Non-metropolitan regions in Croatia',
 'Non-metropolitan regions in Czech Republic',
 'Non-metropolitan regions in Denmark',
 'Non-metropolitan regions in Estonia',
 'Non-metropolitan regions in Finland',
 'Non-metropolitan regions in Germany',
 'Non-metropolitan regions in Greece',
 'Non-metropolitan regions in Hungary',
 'Non-metropolitan regions in Ireland',
 'Non-metropolitan regions in Italy',
 'Non-metropolitan regions in Latvia',
 'Non-metropolitan regions in Lithuania',
 'Non-metropolitan regions in Malta',
 'Non-metropolitan regions in Netherlands',
 'Non-metropolitan regions in North Macedonia',
 'Non-metropolitan regions in Portugal',
 'Non-metropolitan regions in Romania',
 'Non-metropolitan regions in Serbia',
 'Non-metropolitan regions in Slovakia',
 'Non-metropolitan regions in Slovenia',
 'Non-metropolitan regions in Spain',
 'Non-metropolitan regions in Sweden',
 'Non-metropolitan regions in United Kingdom']
countries = ['United Kingdom',"West Midlands urban area","North Macedonia","Austria","Belgium","Bulgaria","Croatia","Cyprus","Czechia","Denmark","Estonia","Finland","France","Germany","Germany (until 1990 former territory of the FRG)","Greece","Hungary","Ireland","Italy","Latvia","Lithuania","Luxembourg","Malta","Netherlands","Poland","Portugal","Romania","Serbia","Slovakia","Slovenia","Spain","Sweden"]
countries.extend(non_metro)

In [ ]:
filtered_data2=filtered_data
for i in countries:
    filtered_data2=filtered_data2[filtered_data2.METROREG!=i]

In [ ]:
import numpy as np
import scipy

In [ ]:
# Preprocessing on the population column
filtered_data2.population = filtered_data2.population.replace(':', np.nan) # replacing NAs (noted ':') by NaN
filtered_data2.population = filtered_data2['population'].replace(r',', '',regex=True)  # removing commas
filtered_data2.population = filtered_data2['population'].astype(float)  #changing column type

# Preprocessing on the GDP column
filtered_data2.gdp = filtered_data2.gdp.replace(':',np.nan )  # replacing missing values (noted ':') by NaN
filtered_data2.gdp = filtered_data2['gdp'].replace(r',', '',regex=True)  # removing commas
filtered_data2.gdp = filtered_data2['gdp'].astype(float)  # changing column type

# Employed_persons column
filtered_data2.employed_persons = filtered_data2.employed_persons.replace(':', np.nan)
filtered_data2.employed_persons = filtered_data2['employed_persons'].replace(r',', '',regex=True)
filtered_data2.employed_persons = filtered_data2['employed_persons'].astype(float)

In [ ]:
filtered_data2[filtered_data2['METROREG'] == 'Bergen']

In [ ]:
from scipy.optimize import curve_fit

def F(x,a,b):
    return a*x+b

def nan_position(k):
    z=np.argwhere(np.isnan(np.array(k)))
    l=[]
    for i in z:
        l.append(i[0])
    return l

def treat_missing(k):
    #cities_removed=[]
    #if not enough values-> drop the city, threshold of 8 missing values per column
    #if all(x <=8 for x in list(k.isnull().sum())) == True:
        #cities_removed.append(v)
    #employed_persons
    missing=k.isnull().sum()
    if missing[2] != 0: #first linear regression
        L1=list(k.employed_persons)
        m1=nan_position(k.employed_persons) #gets position of missing
        X1=[i for i in range(len(list(k.employed_persons))) if str(list(k.employed_persons)[i])!='nan']
        Y1=[i for i in list(k.employed_persons) if str(i)!='nan']
        params1 = curve_fit(F, xdata=X1, ydata=Y1) #performs linear regression, params[0] contains a and b
        for i in m1:
            L1[i] = params1[0][0]*i + params1[0][1]
        k.employed_persons = L1
        
    
    if missing[3] != 0: #second linear regression
        L2=list(k.gdp)
        m2=nan_position(k.gdp) #gets position of missing
        X2=[i for i in range(len(list(k.gdp))) if str(list(k.gdp)[i])!='nan']
        Y2=[i for i in list(k.gdp) if str(i)!='nan']
        params2 = curve_fit(F, xdata=X2, ydata=Y2) #performs linear regression, params[0] contains a and b
        for i in m2:
            L2[i] = params2[0][0]*i + params2[0][1]
        k.gdp=L2 #replacing the column
    
    if missing[4] != 0: #third linear regression
        L3=list(k.population)
        m3=nan_position(k.population) #gets position of missing
        X3=[i for i in range(len(list(k.population))) if str(list(k.population)[i])!='nan']
        Y3=[i for i in list(k.population) if str(i)!='nan']
        params3 = curve_fit(F, xdata=X3, ydata=Y3) #performs linear regression, params[0] contains a and b
        for i in m3:
            L3[i] = params3[0][0]*i + params3[0][1]
        k.population=L3
        print(k.population) #replacing the column
    return

In [ ]:
for v in filtered_data2['METROREG'].unique():
    treat_missing(filtered_data2[filtered_data2['METROREG'] == v])

In [ ]:
filtered_data2

In [ ]:
l=2
if 2 ==l:
    if type(l)==type(3):
        print('hello')

In [ ]:
cities_removed=[]
for v in filtered_data2['METROREG'].unique():
    print(max(list(filtered_data2[filtered_data2['METROREG'] == v].isnull().sum())))
    print(v)
    if max(list(filtered_data2[filtered_data2['METROREG'] == v].isnull().sum()))>8:
        print(max(list(filtered_data2[filtered_data2['METROREG'] == v].isnull().sum())))
        print(filtered_data2[filtered_data2['METROREG'] == v])
        cities_removed.append(v)

In [ ]:
filtered_data

In [ ]:
cities_removed

In [ ]:
max(list(filtered_data2[filtered_data2['METROREG'] == 'Bergen'].isnull().sum()))

In [ ]:
all(x >=9 for x in list(filtered_data2[filtered_data2['METROREG'] == 'Bergen'].isnull().sum()))

In [ ]:
"""according to various sources gdp per capita growth is linear in european countries since 1950 
(Real GDP per capita in developed countries, Ivan O. Kitov)"""

In [ ]:
filtered_data2[350:400]

In [ ]:
TEST = filtered_data2[filtered_data2['METROREG'] == 'Bergen']

In [ ]:
TEST.isnull().sum()

In [ ]:
TEST

In [ ]:
X=[i for i in list(TEST.population) if str(i)!='nan']
Y=[i for i in range(len(list(TEST.population))) if str(list(TEST.population)[i])!='nan']

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.scatter(Y,X)

In [ ]:
import math

In [ ]:
from scipy.optimize import curve_fit
def F(x,a,b):
    return a*x+b

params = curve_fit(F, xdata=Y, ydata=X)

In [ ]:
params

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
z=[i for i in range(20)]

In [ ]:
plt.plot(X,Y)
plt.plot(z,z*params[0][0]**2+params[0][1]*z+params[0][2])

In [ ]:
#Code checking if there are indeed 19 instances per city
for v in filtered_data2['METROREG'].unique():
    print(filtered_data2[filtered_data2['METROREG'] == v])

In [ ]:
[0,0,0].count(0)

In [ ]:
def treat_missing(k,t): #t is a threshold for the number of zeros (mising value we allow)
    L1 = list(k.employed_persons)
    L2 = list(k.gdp)
    L3 = list(k.population) 
    
    if L1.count(0)>t,
    

In [ ]:
#need to check if there are zeros in subdataframe
def treat_missing(k):
    #nested helper function
    #needed because .index() doesn't retrieve position for more than one value
    #eg: for l=[0,0,2], l.index(0) returns 0 and not (0,1)
    def get_zeros(k):
        z=[]
        for i in range(len(k)):
            if k[i]==0:
                z.append(i)
        return z  
    #turns the dataframe into lists
    L1 = list(k.employed_persons)
    L2 = list(k.gdp)
    L3 = list(k.population) 
    z1=get_zeros(L1)
    z2=get_zeros(L2)
    z3=get_zeros(L3)
    #for missing single values at either ends 
    
    #employed_persons
    if len(z1) == 1 and z1[0] == 0:
        L1[0] = L1[1] - abs(L1[1]-L1[2])
        k.employed_persons = L1
    if len(z1) == 1 and z1[0] == 18:
        L1[18] = L1[17] + abs(L1[17]-L1[16])
        k.employed_persons = L1
    
    #gdp
    if len(z2) == 1 and z2[0] == 0:
        L2[0] = L2[1] - abs(L2[1]-L2[2])
        k.gdp = L2
    if len(z2) == 1 and z2[0] == 18:
        L2[18] = L2[17] + abs(L2[17]-L2[16])
        k.gdp = L2
    
    #population
    if len(z3) == 1 and z3[0] == 0:
        L3[0] = L3[1] - abs(L3[1]-L3[2])
        k.population = L3
    if len(z3) == 1 and z3[0] == 18:
        L3[18] = L3[17] + abs(L3[17]-L3[16])
        k.population = L3
    #For missing values at ends (more than one)
    #nested function
    #[0,1]->True
    #[0,2]->False
    def evenly_spaced(l):
        for i in range(len(l)-1):
            if abs(l[i]-l[i+1])!=1:
                return False
        return True

    #employed_persons
    if (len(z1) >1) and evenly_spaced(z1) == True and z1[0] == 0:
        for i in reversed(z1):
            L1[i] = L1[i+1] - abs(L1[i+1] - L1[i+2])
        k.employed_persons = L1
    
    #gdp
    if (len(z2) >1) and evenly_spaced(z2) == True and z2[0] == 0:
        for i in reversed(z2):
            L2[i] = L2[i+1] - abs(L2[i+1] - L2[i+2])
        k.gdp = L2
        
    #population
    if (len(z3) >1) and evenly_spaced(z3) == True and z3[0] == 0:
        for i in reversed(z3):
            L3[i] = L3[i+1] - abs(L3[i+1] - L3[i+2])
        k.population = L3
        
    #other end 
    #employed_persons
    if (len(z1) >1) and evenly_spaced(z1) == True and z1[-1] == 18:
        for i in z1:
            L1[i] = L1[i-1] + abs(L1[i-1] - L1[i-2])
        k.employed_persons = L1
    #gdp
    if (len(z2) >1) and evenly_spaced(z2) == True and z2[-1] == 18:
        for i in z2:
            L2[i] = L2[i-1] + abs(L2[i-1] - L2[i-2])
        k.gdp = L2
    #population
    if (len(z3) >1) and evenly_spaced(z3) == True and z3[-1] == 18:
        for i in z3:
            L3[i] = L3[i-1] + abs(L3[i-1] - L3[i-2])
        k.population = L3
    #If values are missing in the middle of the dataframe: -> apply what Simon showed on excel
    #employed_persons
    if ((len(z1) >1) and evenly_spaced(z1) == True) and (z1[-1] != 18 and z1[0] != 0):
        p = L1[z1[0]-1] - L1[z1[-1]+1]
        add = p/(len(z1)+1)
        for i in z1:
            L1[i]=L1[i-1]+add
        k.employed_persons=L1
    #gdp
    if ((len(z2) >1) and evenly_spaced(z2) == True) and (z2[-1] != 18 and z2[0] != 0):
        print(L2)
        print(z2)
        p = L2[z2[0]-1] - L2[z2[-1]+1]
        add = p/(len(z2)+1)
        for i in z2:
            L2[i]=L2[i-1]+add
        k.gdp=L2
    #population
    if ((len(z3) >1) and evenly_spaced(z3) == True) and (z3[-1] != 18 and z3[0] != 0):
        p = L3[z3[0]-1] - L3[z3[-1]+1]
        add = p/(len(z3)+1)
        for i in z3:
            L3[i]=L3[i-1]+add
        k.population=L3
    #more complicated cases
    #0xxx000
    #000xxx0
    #000xxx000
    #3rd helper function for 
    def get_sep(l):
        for i in range(len(l)-1):
            if abs(l[i]-l[i+1])!=1:
                t=(l[i],l[i+1])
        return t
    #0xxx000
    if evenly_spaced(z1) == False and len(z1)>1 and z1[0]==0:
        L1[0] = L1[1] - abs(L1[1]-L1[2])
        p = L1[z1[0]-1] - L1[z1[-1]+1]
        add = p/(len(z1))
        for i in z1[get_sep(z1)[1]:]:
            L1[i]=L1[i-1]+add
        k.employed_persons=L1
    
    #gdp
    if evenly_spaced(z2) == False and len(z2)>1 and z1[0]==0:
        L2[0] = L2[1] - abs(L2[1]-L2[2])
        p = L2[z2[0]-1] - L2[z2[-1]+1]
        add = p/(len(z2))
        for i in z2[get_sep(z2)[1]:]:
            L2[i]=L2[i-1]+add
        k.gdp=L2
    
    #population
    if evenly_spaced(z3) == False and len(z3)>1 and z3[0]==0:
        L3[0] = L3[1] - abs(L3[1]-L3[2])
        p = L3[z3[0]-1] - L3[z3[-1]+1]
        add = p/(len(z3))
        for i in z3[get_sep(z3)[1]:]:
            L3[i]=L3[i-1]+add
        k.population=L3
    
    #000xxx0
    if evenly_spaced(z1) == False and len(z1)>1 and z1[-1]==18:
        L1[18] = L1[17] + abs(L1[17]-L1[16])
        p = L1[z1[0]-1] - L1[z1[-1]+1]
        add = p/(len(z1))
        for i in z1[:get_sep(z1)[1]]:
            L1[i]=L1[i-1]+add
        k.employed_persons=L1
    
    #gdp
    if evenly_spaced(z2) == False and len(z2)>1 and z1[-1]==18:
        L2[0] = L2[1] - abs(L2[1]-L2[2])
        p = L2[z2[0]-1] - L2[z2[-1]+1]
        add = p/(len(z2))
        for i in z2[get_sep(z2)[1]:]:
            L2[i]=L2[i-1]+add
        k.gdp=L2
    
    #population
    if evenly_spaced(z3) == False and len(z3)>1 and z3[-1]==18:
        L3[0] = L3[1] - abs(L3[1]-L3[2])
        p = L3[z3[0]-1] - L3[z3[-1]+1]
        add = p/(len(z3))
        for i in z3[get_sep(z3)[1]:]:
            L3[i]=L3[i-1]+add
        k.population=L3
    
    
        
        
        
        
        
                
        
        
    
    return

In [ ]:
get_sep([1,2,4])[0]

In [ ]:
def get_sep(l):
    for i in range(len(l)-1):
        if abs(l[i]-l[i+1])!=1:
            t=(l[i],l[i+1])
    return t

In [ ]:
 Yo=filtered_data2[filtered_data2['METROREG'] == 'Bergen']

In [ ]:
treat_missing(Yo)

In [ ]:
Yo

In [ ]:
for v in filtered_data2['METROREG'].unique():
    treat_missing(filtered_data2[filtered_data2['METROREG'] == v])

In [ ]:
filtered_data2[20:]

In [ ]:
filtered_data3 = filtered_data2[(filtered_data2['TIME']>=2000) & (filtered_data2['population']>500000) & (filtered_data2['gdp']>40000)]

In [ ]:
filtered_data3 = filtered_data3[filtered_data3['METROREG'].apply(lambda x: len(x) < 27)]

In [ ]:
import geopy
from geopy.geocoders import Nominatim

In [ ]:
geolocator = Nominatim(user_agent="Master-Student")
def get_coord(k):
    assert type(k)==str
    return (geolocator.geocode(k).latitude,geolocator.geocode(k).longitude)

In [ ]:
latlong=[0 for i in range(filtered_data3.shape[0])]

In [ ]:
filtered_data3['latitude'] = latlong
filtered_data3['longitude'] = latlong

In [ ]:
filtered_data3

In [ ]:
d={}
for i in sorted(set(list(filtered_data3.METROREG))):
    print(i)
    d[i]=get_coord(i)

In [ ]:
for i in range(len(list(d.keys()))):
    for idx, row in filtered_data3.iterrows():
            filtered_data3.loc[idx,'latitude'] = list(d.values())[i][0]
            filtered_data3.loc[idx,'longitude'] = list(d.values())[i][1]

In [ ]:
d2=dict(filtered_data3['METROREG'].value_counts())

In [ ]:
d2

In [ ]:
for i in d2.keys():
    if d2[i]<10:
        filtered_data3=filtered_data3[filtered_data3.METROREG!=i]

In [ ]:
filtered_data3